In [3]:
# https://www.kaggle.com/code/tabrom/group3-panda-tiling/notebook
# Based on https://www.kaggle.com/code/iafoss/panda-16x128x128-tiles

### Per WSI, create a square image of N tiles, based on number of tissue pixels (and their darkness).¶

In [4]:
# The path can also be read from a config file, etc.
OPENSLIDE_PATH = r'C:\Users\johng\Downloads\openslide-win64-20231011\bin'

import os
if hasattr(os, 'add_dll_directory'):
    # Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide
    
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt 
import PIL
from IPython.display import Image, display
import skimage.io
# import tifffile
from tqdm.notebook import tqdm
import zipfile
import cv2 as cv

In [5]:
# Location of the files
data_dir = 'train_images'
mask_dir = 'train_label_masks/'
out_dir = 'tiled_images/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
# Location of training labels
samples = pd.read_csv('train.csv')

In [6]:
## Takes an openslide object and returns the top left coordinates of N tiles (of a given size) with the most tissue pixels. 

# Note: slide.level_dimensions[level] = (width,height).
# Note: padding is done to the right and bottom, this is to keep it simple while having at most 1 tile in memory at a time.
def get_tile_locations_from_slide(slide, tile_size, N, level):
    tiles = []
    required_padding = False
    xlocs, ylocs = np.arange(0, slide.level_dimensions[level][0], tile_size), np.arange(0, slide.level_dimensions[level][1], tile_size) # Get the coordinates of the top left corners of the tiles.
    for x_i, xloc in enumerate(xlocs):
        for y_i, yloc in enumerate(ylocs):
            region = np.copy(slide.read_region((xloc*(4**level),yloc*(4**level)), level, (tile_size,tile_size))) # The position is wrt. level 0, so must convert to level 0 coordinates by multiplying by the downsampling factor.
            region_arr = np.asarray(region)[:,:,:3] # Ignore the alpha channel
            if xloc+tile_size > slide.level_dimensions[level][0] or yloc+tile_size > slide.level_dimensions[level][1]: # if the tile goes out of bounds
                region_arr[region_arr==0] = 255
                required_padding = True
            pixel_sum = region_arr.sum()
            tiles.append({'xloc': xloc, 'yloc': yloc, 'pixel_sum': pixel_sum, 'required_padding': required_padding}) # store top left corner location and the tile's pixel_sum
            required_padding = False
    sorted_tiles = sorted(tiles, key= lambda d: d['pixel_sum']) # Sort tiles based on their pixel_sum field
    sorted_tiles = sorted_tiles[:N] # Get top N tiles
    return sorted_tiles

In [7]:
# Creates a single image (array) from the selected tiles
def create_tiled_image(slide, tiles, tile_size, N_tiles, level):
    N_side = int(np.sqrt(N_tiles)) # How many tiles is the image wide/tall
    tiled_image = np.ones((N_side*tile_size,N_side*tile_size,3), dtype=np.uint8)*255
    for i, tile in enumerate(tiles):
        region = np.copy(np.asarray(slide.read_region((tile['xloc']*(4**level),tile['yloc']*(4**level)), level, (tile_size,tile_size)))) # The position is wrt. level 0, so must convert to level 0 coordinates by multiplying by the downsampling factor.
        if tile['required_padding']:
            region[region==0] = 255
        tiled_image[tile_size*(i//(N_side)):tile_size*(i//(N_side))+tile_size, tile_size*(i%(N_side)) : tile_size*(i%(N_side))+tile_size, :] = region[:,:,:3]
    return tiled_image

In [8]:
def save_img(img, filename, folder):
    if not os.path.exists(output_folder):
        os.makedirs(folder)
    encoded_img = cv.imencode('.png', img)[1]
    print(f'Saving to: {os.path.join(folder, filename+"_tiled.png")}')
    cv.imwrite(os.path.join(folder, filename+"_tiled.png"), img)
    
def save_img_to_zip(img, filename, zip_file):
    encoded_img = cv.imencode('.png', img)[1]
    zip_file.writestr(filename+'_tiled.png', encoded_img)

In [9]:
## Plots the selected slices over the original slide.
def show_tile_locations(slide, tiles, tile_size, level, is_mask=False, data_provider='radboud'):
    wsi = slide.read_region((0,0), level, slide.level_dimensions[level]) # Get whole slide image pixel values
    wsi = np.asarray(wsi)[:,:,:3] # Convert to array (optional) and ignore alpha channel
    if is_mask:
        wsi = color_code_mask(wsi,data_provider, level)
    fig, ax = plt.subplots()
    ax.imshow(wsi)
    for tile in tiles: # Draw the tiles
        rect = matplotlib.patches.Rectangle((tile['xloc'],tile['yloc']),tile_size,tile_size, linewidth=1, edgecolor='b', facecolor='none')
        ax.add_patch(rect)
    plt.show()

In [10]:
def color_code_mask(mask, data_provider:str, level:int):
    mask = np.copy(mask)
    # Set background to white
    mask[mask==0] = 255
    mask_data = mask[:,:,0]
    if data_provider == 'radboud':
        # Stroma
        mask[:,:,:][mask_data==1] = 200
        # healthy/Benign
        mask[:,:,:][mask_data==2] = 150
        # Cancerous (Gleason 3)
        mask[:,:,1:3][mask_data==3] = 75
        mask[:,:,0][mask_data==3] = 170
        # Cancerous (Gleason 4)
        mask[:,:,1:3][mask_data==4] = 25
        mask[:,:,0][mask_data==4] = 200
        # Cancerous (Gleason 5)
        mask[:,:,1:3][mask_data==5] = 0
        mask[:,:,0][mask_data==5] = 255
    elif data_provider == 'karolinska':
        # Benign
        mask[:,:,:][mask_data==1] = 200
        # Cancerous
        mask[:,:,1:3][mask_data==2] = 75
        mask[:,:,0][mask_data==2] = 255
    return mask

In [11]:
## Plots the selected slices over the original slide.
def show_tile_locations(slide, tiles, tile_size, level, is_mask=False, data_provider='radboud'):
    wsi = slide.read_region((0,0), level, slide.level_dimensions[level]) # Get whole slide image pixel values
    wsi = np.asarray(wsi)[:,:,:3] # Convert to array (optional) and ignore alpha channel
    if is_mask:
        wsi = color_code_mask(wsi,data_provider, level)
    fig, ax = plt.subplots()
    ax.imshow(wsi)
    for tile in tiles: # Draw the tiles
        rect = matplotlib.patches.Rectangle((tile['xloc'],tile['yloc']),tile_size,tile_size, linewidth=1, edgecolor='b', facecolor='none')
        ax.add_patch(rect)
    plt.show()

In [12]:
## Plots the selected slices over the original slide.
def save_tile_locations_plots(slide, mask, tiled_image, tiles, tile_size, level, data_provider, isup_grade, file_name, show=False):
    wsi = slide.read_region((0,0), level, slide.level_dimensions[level]) # Get whole slide image pixel values
    wsi = np.asarray(wsi)[:,:,:3] # Convert to array (optional) and ignore alpha channel
    mask = mask.read_region((0,0), level, slide.level_dimensions[level])
    mask = np.asarray(mask)[:,:,:3] # Convert to array (optional) and ignore alpha channel
    mask = color_code_mask(mask,data_provider, level)
    fig, (ax_slide, ax_mask, ax_tiled_image) = plt.subplots(1,3, figsize=(22,10))
#     plt.suptitle(f"ISUP grade: {isup_grade}")
    ax_slide.imshow(wsi)
    ax_slide.set_title("WSI with tiles")
    for tile in tiles: # Draw the tiles
        rect = matplotlib.patches.Rectangle((tile['xloc'],tile['yloc']),tile_size,tile_size, linewidth=1, edgecolor='b', facecolor='none')
        ax_slide.add_patch(rect)
    ax_mask.imshow(mask)
    ax_mask.set_title("Mask with tiles")
    for tile in tiles: # Draw the tiles
        rect = matplotlib.patches.Rectangle((tile['xloc'],tile['yloc']),tile_size,tile_size, linewidth=1, edgecolor='b', facecolor='none')
        ax_mask.add_patch(rect)
    ax_tiled_image.imshow(tiled_image)
    ax_tiled_image.set_title("Extracted tiles")
    if show:
        plt.show()
    plt.savefig(file_name+".png", facecolor='white', transparent=False)
    plt.close()

In [13]:
# THE PARAMETERS 🔥
N_tiles = 6**2 # Number of tiles per image, should have a whole square root
tile_size = 2**8 # Width/height of tile, 2**8 = 256
level = 1 # 0 is highest resolution, 2 is lowest resolution, good compromise is level 1
output_folder = os.path.join(out_dir, 'tiled_images')
plot_output = False
Y = 25
selected_indices = np.zeros(Y*6)
for i, isup_grade in enumerate(range(0,6)):
    isup_indices = list(samples[samples['isup_grade']==isup_grade].index)
    selected_indices[Y*i:Y*(i+1)]= np.random.choice(isup_indices, Y, replace=False)
    

for indx in tqdm(selected_indices):
    sample = samples.iloc[int(indx)]
    file_name = sample.loc['image_id']
    data_provider = sample.loc['data_provider']
    slide = openslide.OpenSlide(os.path.join(data_dir, file_name+'.tiff'))
    mask = openslide.OpenSlide(os.path.join(mask_dir, file_name+'_mask.tiff'))

    tiles = get_tile_locations_from_slide(slide, tile_size, N_tiles, level) # Get tile coordinates of top N tiles
    tiled_image = create_tiled_image(slide, tiles, tile_size, N_tiles, level) # Convert the tiles information into a tiled image

    isup_grade = sample.loc['isup_grade']
    save_path = os.path.join(out_dir, file_name+f"_isup{isup_grade}")
    save_tile_locations_plots(slide, mask, tiled_image, tiles, tile_size, level, 
                              data_provider=data_provider, isup_grade=isup_grade, file_name=save_path ,show=plot_output);

#     if plot_output:
#         show_tile_locations(slide, tiles, tile_size, level) # Plot the tiles over the original slide
#         show_tile_locations(mask, tiles, tile_size, level, is_mask=True, data_provider=data_provider) # Plot the tiles over the original mask

    slide.close()

  0%|          | 0/150 [00:00<?, ?it/s]

OpenSlideUnsupportedFormatError: Unsupported or missing image file

In [ ]:
# !zip -r tile_placements.zip /kaggle/working/tiled_images/
# from IPython.display import FileLink 
# os.chdir(r'/kaggle/working')
# FileLink(r'tile_placements.zip')